In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def extract_and_clean_chmi_excel_sheet(excel_data, sheet_name):
    """Parse ČHMÚ historical meteo excel data"""
    # načti list z excel souboru a převeď na tidy data formát
    data_tidy = (
        excel_data.parse(sheet_name, skiprows=3)
        .melt(id_vars=["rok", "měsíc"], var_name="den", value_name=sheet_name)
        .dropna()
    )
    # vytvoř časovou řadu datumů
    datum = pd.to_datetime(
        data_tidy[["rok", "měsíc", "den"]].rename(
            columns={"rok": "year", "měsíc": "month", "den": "day"}
        )
    )
    # přidej sloupec datum jako index a odstraň den, měsíc, rok a vrať setříděný výsledek
    return (
        data_tidy.assign(datum=datum)
        .set_index("datum")
        .drop(columns=["rok", "měsíc", "den"])
        .sort_index()
    )

In [ ]:
# otevři Excel soubor
excel_data_ruzyne = pd.ExcelFile("P1PRUZ01.xls")
# načti všechny listy kromě prvního
extracted_sheets = (
    extract_and_clean_chmi_excel_sheet(excel_data_ruzyne, sheet_name)
    for sheet_name in excel_data_ruzyne.sheet_names[1:]
)
# spoj všechny listy do jednoho DataFrame
ruzyne_tidy = pd.concat(extracted_sheets, axis=1)

In [ ]:
leg_length = pd.DataFrame({"left": [81, 81.4], "right": [78.2, 78]})
leg_length

**Úkol:** Má `leg_length` podobu tidy data? Pokud ne, dokážeš tato data uspořádat správně?

In [ ]:
leg_length.melt(value_name="leg length", var_name="leg side")

**Úkol:** `to_datetime` dokáže pracovat i s řetězci, což se často hodí. Převeďte `ladies_times` na vohodný typ pro časové údaje, přiřaďte naši časovou zónu a poté pomocí `tz_convert` převeďte na UTC. Možná budete muset pandám vysvětlit, že v Česku jsou v datumech nejdříve dny, na rozdíl třeba od Ameriky. Naštěstí na to stačí jeden jednoduchý argument pro `to_datetime`.

In [ ]:
ladies_times = ["23. 1. 2020 18:00", "30. 1. 2020 18:00", "6. 2. 2020 18:00"]

In [ ]:
pd.to_datetime(ladies_times, dayfirst=True).tz_localize("Europe/Prague").tz_convert("UTC")

**Úkol:** Zařaďte `dropna` do sestrojení datumů tak, abychom nemuseli použít `errors="coerce"` pro `to_datetime`.

In [ ]:
# řešení
pd.to_datetime(
    teplota_prumerna_tidy.dropna()[["rok", "měsíc", "den"]].rename(
        columns={"rok": "year", "měsíc": "month", "den": "day"}
    ),
)

**Úkol**: Vytvořte `teplota_prumerna_tidy_clean_indexed` se sloupcem `datum` jako indexem a bez sloupců `rok`, `měsíc` a `den`. Můžete použít metodu `drop`.

In [ ]:
# řešení
teplota_prumerna_tidy_clean_indexed = teplota_prumerna_tidy.set_index("datum").drop(
    columns=["rok", "měsíc", "den"]
)

**Úkol:** Jaká byla průměrná teplota první (a jedinou) neděli v roce 2010, která byla zároveň začátkem měsíce? Pokud máte řešení a čas, zkuste vymyslet aternativní způsob(y). 

In [ ]:
# řešení 1
ruzyne_tidy.loc[
    ruzyne_tidy.index.is_month_start
    & (ruzyne_tidy.index.weekday == 6)
    & (ruzyne_tidy.index.year == 2010),
    "teplota průměrná",
]

**Cvičení**: Doplňte vytvoření pomocných sloupců (jistě uhádnete jakého pandas-typu bude :) `season` a `significant_precipitation`. První definuje roční období (jen jednoduše podle kalendářních měsíců), drůhá označuje dny kdy byly srážky vyšší než v **90 %** všech dní v našich datech (můžete zkusit i jiný limit).

1. Porovnejte numericky základní statistiky celého data setu a podmnožiny, kdy výrazně pršelo nebo sněžilo? Zvyšují v průměru srážky teplotu?
2. Použijte `sns.catplot` pro vizuální srovnící distribučních funkcí pro jednotlivá roční období a dny s málo / hodně srážkami.

In [ ]:
season = ruzyne_tidy.index.month.map({
    1: "zima",
    2: "zima",
    3: "jaro",
    4: "jaro",
    5: "jaro",
    6: "léto",
    7: "léto",
    8: "léto",
    9: "podzim",
    10: "podzim",
    11: "podzim",
    12: "zima",
})

significant_precipitation = ruzyne_tidy["úhrn srážek"] > ruzyne_tidy["úhrn srážek"].quantile(0.9)

In [ ]:
(ruzyne_tidy.loc[significant_precipitation]
 .describe()
) - \
ruzyne_tidy.describe()

In [ ]:
(ruzyne_tidy.loc[significant_precipitation]
 .agg(["mean", "max", "min", "std"])
) - \
ruzyne_tidy.agg(["mean", "max", "min", "std"])

In [ ]:
sns.catplot(
    data=ruzyne_tidy.assign(
        significant_precipitation=significant_precipitation, 
        season=season,
    ),
    kind="box",
    aspect=2,
    hue="significant_precipitation",
    y="teplota průměrná",
    x="season",
);

**Úkol:** Navrhněte vhodnou agregaci pro maximální teplotu (ne průměr) a vykreslete.

In [ ]:
# řešení
ruzyne_yearly["teplota maximální"].max().plot();

**Úkol:** Převzorkujte údaje za rok 2018 po měsících. Jaký měsíc měl nejvíc srážek, tj. za jaký byl součet sloupce "úhrn srážek" nejvyšší?

In [ ]:
# řešení
ruzyne_tidy.loc[ruzyne_tidy.index.year == 2018, "úhrn srážek"].resample(
    "1M"
).sum().sort_values(ascending=False).index[0].month

## TODO - skladiště nápadů

* Rozdělení na horké / studené dny -> cut?
* Jak vypadá rozdělovací funkce pro dny kdy pršelo? Jaká je střední hodnota, medián, Q1, Q3 a směrodatná odchylka.
* Kolik dní za sebou nejdéle pršelo?

...


**Úkol (damácí?):** Načtěte data ... *TODO připravit soubor schibami* ..., kde jsou chybějící data označena jako `#CHYBA MĚŘENÍ`. Odstraňte pouze řádky, kde je špatné datum. Pomocí `fillna` pak nahraďte chybějící měření poslední předchozí hodnotou.

# **TODO** Domácí úkoly

* Ověřit pranostiky: Medardova kápě, ledoví muži, vyberte si pár vlastních, např. z https://www.meteocentrum.cz/zajimavosti/pranostiky/.
* Kolik dní v kuse nejdéle pršelo / sněžilo? 1) V Ruzyni, 2) ve všech meteostanicích? Kolik naopak nepršelo? A kdy to bylo?

## Možná data

### Výroba elektřiny

In [ ]:
GENERATION_FILENAME = "generation_2010.csv.gz"
GENERATION_URL = f"https://data4pydata.s3-eu-west-1.amazonaws.com/pyladies/{GENERATION_FILENAME}"

In [ ]:
save_file_from_url(GENERATION_URL, GENERATION_FILENAME)

In [ ]:
generation_2010 = pd.read_csv("generation_2010.csv.gz")

In [ ]:
generation_2010.sample(10)

### Ceny elektřiny

In [ ]:
import zipfile

In [ ]:
def get_ote(year: int) -> pd.DataFrame:
    filename = f"Rocni_zprava_o_trhu_{year}_V2.zip"
    url = f"https://data4pydata.s3-eu-west-1.amazonaws.com/pyladies/{filename}"
    save_file_from_url(url, filename)
    with zipfile.ZipFile(filename) as archive:
        with archive.open(archive.namelist()[0]) as xls_file:
            ote_data = pd.read_excel(
                xls_file, sheet_name="DT ČR", usecols="A:K", header=5
            )

    return ote_data

In [ ]:
ote_2018 = get_ote(2010)

In [ ]:
ote_2018.sample(10)

**Úlohy:**

* Převeďte na správný čas v časové zóně "Europe/Prague".
* Spojit soubory
* Jak se liší průměrné a maximální ceny v kalendářních dnech? Je nějaký den výrazně levnější?

**TODO**